# LITTLE LEMON DATABASE
Little Lemon needs to build a robust relational database system in MySQL in which they can store large amounts of data. They then need to easily manage and find this data as required. This database system should maintain information about the following aspects of the business:

* Customers.
* Menus.
* Bookings.
* Staff.
* Orders.
* Deliver Status.

### ER diagram and it's implementation

First we need to create a normalized ER diagram (that adheres to 1NF, 2NF and 3NF) with relevant relationships to meet the data requirements of Little Lemon. When creating your diagram, include the following tables:

* Bookings: To store information about booked tables in the restaurant including booking id, date and table number.
* Orders: To store information about each order such as order date, quantity and total cost.
* Order delivery status: To store information about the delivery status of each order such as delivery date and status.
* Menu: To store information about cuisines, starters, courses, drinks and desserts.
* Customer details: To store information about the customer names and contact details.
* Staff information: Including role and salary.

Using MySQL Workbench we create the following ER diagram:

![LittleLemon_ER_Diagram.png](LittleLemon_ER_Diagram.png)

Now, using the forward engineer method in MySQL Workbench we can implement the previous ER diagram inside MySQL server, and it will look like:

![LittleLemonDB.png](LittleLemonDB.png)

### Tables to populate the LittleLemonDB
Now we need to populate the database, and for that, we will use the `LittleLemonDB.csv` file to create different tables for this purpose.

In [28]:
import pandas as pd
import random
import mysql.connector as connector
from mysql.connector import Error, errorcode

In [29]:
# Let's create a dataframe from the LittleLemonDB.csv file
littlelemondb = pd.read_csv('LittleLemonDB.csv', sep=';')
littlelemondb

,OrderID,OrderDate,DeliveryDate,CustomerID,CustomerName,City,Country,PostalCode,CountryCode,Cost,Sales,Quantity,Discount,DeliveryCost,CourseName,CuisineName,StarterName,DesertName,Drink,Sides
0,54-366-6861,15/06/2020,26/03/2020,72-055-7985,Laney Fadden,Daruoyan,China,993-0031,CN,125,"187,5",2,20,"60,51",Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,63-761-3686,25/08/2020,17/07/2020,65-353-0657,Giacopo Bramich,Ongjin,North Korea,216282,KP,235,"352,5",1,15,"96,75",Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,65-351-6434,17/08/2021,24/04/2020,90-876-6799,Lia Bonar,Quince Mil,Peru,663246,PE,75,"112,5",3,"10,52","36,37",Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta
3,36-917-2834,14/08/2021,13/04/2020,73-873-4827,Merrill Baudon,Susaki,Japan,987-0352,JP,220,330,3,"11,23","5,49",Carbonara,Turkish,Tomato bread,Affogato,Roma Red wine,Focaccia
4,86-114-9232,20/12/2020,2/02/2021,80-927-5246,Tasia Fautly,Tobruk,Libya,351 01,LY,320,480,2,"51,05","63,64",Kabasa,Greek,Falafel,Turkish yoghurt,Ankara White Wine,Meatballs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,32-698-1958,2/11/2021,12/10/2020,79-834-1150,Paton Fritzer,Nowe Grocholice,Poland,993-1266,PL,"124,62","186,93",3,"9,71","49,46",Shwarma,Turkish,Flatbread,Baklava,Turkish Coffee,Fries
20996,66-068-8313,12/10/2021,20/10/2021,91-484-8549,Kelbee Illesley,Sete Lagoas,Brazil,1104217362,BR,"144,09","216,135",2,"80,34","28,02",Greek salad,Greek,Minestrone,Greek yoghurt,Athens White wine,Tapas
20997,02-311-4674,30/06/2022,4/11/2022,58-358-2956,Christabella Coathup,Bangil,Indonesia,1104664326,ID,"101,6","152,4",1,"54,08","9,48",Bean soup,Italian,Tomato bread,Ice cream,Corfu Red Wine,Potato salad
20998,85-569-0097,21/12/2020,5/10/2020,40-946-4285,Ephrem Steers,Saint-André-Avellin,Canada,987-1587,CA,"93,13","139,695",2,"98,01","43,62",Pizza,Italian,Falafel,Cheesecake,Italian Coffee,Bruschetta


For the **Customers** table we'll use the columns `CustomerID`, `CustomerName`, `Country`, `CountryCode`, `City`, and `PostalCode` from the `littlelemondb` dataframe. Also, if we take a look at the PostalCode in that file we'll notice that some columns start with the `'-'` sign and others have decimal values, so we need to remove those.

In [30]:
# Dropping the customer duplicates
Customers = littlelemondb[['CustomerID', 'CustomerName', 'Country', 'CountryCode', 'City', 'PostalCode']].drop_duplicates(subset=['CustomerID'])

# Removing quotes, the '-' sign at the beggining of PostalCode column and the decimal points
Customers['PostalCode'].replace(r"\"\'", '', regex=True, inplace=True)
Customers['PostalCode'].replace(r'^-', '', regex=True, inplace=True)
Customers['PostalCode'].replace(r',.*$', '', regex=True, inplace=True)

Customers

,CustomerID,CustomerName,Country,CountryCode,City,PostalCode
0,72-055-7985,Laney Fadden,China,CN,Daruoyan,993-0031
1,65-353-0657,Giacopo Bramich,North Korea,KP,Ongjin,216282
2,90-876-6799,Lia Bonar,Peru,PE,Quince Mil,663246
3,73-873-4827,Merrill Baudon,Japan,JP,Susaki,987-0352
4,80-927-5246,Tasia Fautly,Libya,LY,Tobruk,351 01
...,...,...,...,...,...,...
995,79-834-1150,Paton Fritzer,Poland,PL,Nowe Grocholice,649-6384
996,91-484-8549,Kelbee Illesley,Brazil,BR,Sete Lagoas,33677
997,58-358-2956,Christabella Coathup,Indonesia,ID,Bangil,649-6384
998,40-946-4285,Ephrem Steers,Canada,CA,Saint-André-Avellin,353773


For the **Menus** table we're going to combine the columns `CourseName`, `CuisineName`, `StarterName`, `DesertName`, `Drink` and `Sides` from the `littlelemondb` dataframe, assuming that this will configure an *unique menu*, and use that as `MenuID`.

In [31]:
Menus = littlelemondb[['CourseName', 'CuisineName', 'StarterName', 'DesertName', 'Drink', 'Sides']].drop_duplicates()

# Creating a menuID
menuID = [x for x in range(1, len(Menus)+1)]
Menus.insert(0,'MenuID', menuID)

Menus

,MenuID,CourseName,CuisineName,StarterName,DesertName,Drink,Sides
0,1,Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,2,Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,3,Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta
3,4,Carbonara,Turkish,Tomato bread,Affogato,Roma Red wine,Focaccia
4,5,Kabasa,Greek,Falafel,Turkish yoghurt,Ankara White Wine,Meatballs
...,...,...,...,...,...,...,...
102,68,Greek salad,Italian,Flatbread,Greek yoghurt,Athens White wine,Tapas
103,69,Bean soup,Turkish,Minestrone,Ice cream,Corfu Red Wine,Potato salad
104,70,Pizza,Greek,Tomato bread,Cheesecake,Italian Coffee,Bruschetta
105,71,Carbonara,Italian,Falafel,Affogato,Roma Red wine,Focaccia


For the **Bookings** table we're going to use the columns `CustomerID` and the `OrderDate` from the `littlelemondb` dataframe, assuming that a customer can make different bookings. Also, we'll create the columns `BookingID` and `TableNumber`, the table number will be assigned in a 'random' way, and the booking id will be set in an orderly way.

In [32]:
Bookings = littlelemondb[['CustomerID', 'OrderDate']].drop_duplicates()

# Creating a bookingID and assigning a random table number
BookingID = [x for x in range(1, len(Bookings)+1)]
TableNumber = [random.randint(1,15) for _ in range(1, len(BookingID)+1)]
Bookings.insert(0,'BookingID', BookingID)
Bookings.insert(len(Bookings.columns),'TableNumber', TableNumber)

Bookings

,BookingID,CustomerID,OrderDate,TableNumber
0,1,72-055-7985,15/06/2020,6
1,2,65-353-0657,25/08/2020,5
2,3,90-876-6799,17/08/2021,7
3,4,73-873-4827,14/08/2021,3
4,5,80-927-5246,20/12/2020,13
...,...,...,...,...
995,996,79-834-1150,2/11/2021,6
996,997,91-484-8549,12/10/2021,10
997,998,58-358-2956,30/06/2022,12
998,999,40-946-4285,21/12/2020,5


For the **Staff** table there are not information in the `littlelemonBD.csv` file, so let's create a dictionary with some random information.

In [33]:
Staff_dict = {
    'StaffID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'StaffName':['Saoirse Conley', 'Marvin Copeland', 'Dayana Reilly', 'Alvaro Stevens', 'Katherine Aguilar', 'Milo Colon', 'Remy McCullough',
            'Briar Allison', 'Chelsea Carroll', 'Oscar Chase', 'Angie Lee', 'Jack Dunlap', 'Iliana Hardin', 'Hassan Solis', 'Miracle Russo'],
    'Role':['Manager', 'Chef', 'SousChef', 'SousChef', 'SousChef', 'Cashier', 'Cashier', 'AuxCook', 'AuxCook', 'AuxCook', 'Waitress',
            'Waiter', 'Waitress', 'Waiter', 'Waiter'],
    'Salary':[5000, 4500, 3500, 3500, 3500, 2300, 2300, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]
    }
Staff = pd.DataFrame(Staff_dict)
Staff

,StaffID,StaffName,Role,Salary
0,1,Saoirse Conley,Manager,5000
1,2,Marvin Copeland,Chef,4500
2,3,Dayana Reilly,SousChef,3500
3,4,Alvaro Stevens,SousChef,3500
4,5,Katherine Aguilar,SousChef,3500
5,6,Milo Colon,Cashier,2300
6,7,Remy McCullough,Cashier,2300
7,8,Briar Allison,AuxCook,2000
8,9,Chelsea Carroll,AuxCook,2000
9,10,Oscar Chase,AuxCook,2000


For the **DeliveryStatus** table we're going to use the columns `OrderDate` and `DeliveryDate` from the `littlelemondb` dataframe. Also, we'll create a `DeliveryStatusID` that will be set in an orderly way.

In [34]:
DeliveryStatus = littlelemondb[['OrderDate', 'DeliveryDate']].drop_duplicates()

# Creating a DeliveryStatusID
DeliveryStatusID = [x for x in range(1, len(DeliveryStatus)+1)]
DeliveryStatus.insert(0,'DeliveryStatusID', DeliveryStatusID)

DeliveryStatus

,DeliveryStatusID,OrderDate,DeliveryDate
0,1,15/06/2020,26/03/2020
1,2,25/08/2020,17/07/2020
2,3,17/08/2021,24/04/2020
3,4,14/08/2021,13/04/2020
4,5,20/12/2020,2/02/2021
...,...,...,...
995,996,2/11/2021,12/10/2020
996,997,12/10/2021,20/10/2021
997,998,30/06/2022,4/11/2022
998,999,21/12/2020,5/10/2020


For the **Orders** table we're going to use the columns `OrderID`, `OrderDate`, `Quantity`, `Cost`, `Discount`, `Sales`, `DeliveryCost`, `DeliveryCost`, `CourseName`, `CuisineName`, `StarterName`, `DesertName`, `Drink` and `Sides` from the `littlelemondb` dataframe. Also, we're going to make some merges with the previous dataframes and create a temporary dataframe to assign in a 'random' way the staff id.

In [35]:
Orders = (littlelemondb[['CustomerID','OrderID', 'OrderDate', 'DeliveryDate', 'Quantity', 'Cost', 'Discount', 'Sales', 'DeliveryCost',
                        'CourseName', 'CuisineName', 'StarterName', 'DesertName', 'Drink', 'Sides']])

# Removing blank spaces and replacing ',' by '.' for decimal cast.
Orders = Orders.replace(r',', '.', regex=True)

# Getting the BookingID and Table Number from the Bookings table
Orders = Bookings.merge(Orders, on= ['CustomerID', 'OrderDate'])
# Getting the MenuID from the Menus table
Orders = Menus.merge(Orders, on=['CourseName', 'CuisineName', 'StarterName', 'DesertName', 'Drink', 'Sides'])
# Getting the DeliveryStatusID from the DeliveryStatus table
Orders = DeliveryStatus.merge(Orders, on= ['OrderDate', 'DeliveryDate'])

# Temporary dataframe to assign in a 'random' way the staff for every order
temp = littlelemondb[['CustomerID','OrderID']].drop_duplicates()
temp.insert(0, 'StaffID', [random.randint(11,15) for _ in range(1, len(temp)+1)])
Orders = temp.merge(Orders, on=['CustomerID', 'OrderID'])

# Getting the final dataframe
Orders = (Orders[['OrderID', 'MenuID', 'BookingID', 'StaffID', 'DeliveryStatusID', 'OrderDate', 'TableNumber', 'Quantity', 'Cost', 'Discount',
                'DeliveryCost', 'Sales']].rename(columns={'OrderID': 'OrderRef'}))

Orders

,OrderRef,MenuID,BookingID,StaffID,DeliveryStatusID,OrderDate,TableNumber,Quantity,Cost,Discount,DeliveryCost,Sales
0,54-366-6861,1,1,14,1,15/06/2020,6,2,125,20,60.51,187.5
1,54-366-6861,1,1,14,1,15/06/2020,6,2,93.13,9.71,60.51,139.695
2,54-366-6861,1,1,14,1,15/06/2020,6,2,75,10,60.51,112.5
3,54-366-6861,1,1,14,1,15/06/2020,6,2,144.09,31.2,60.51,216.135
4,54-366-6861,1,1,14,1,15/06/2020,6,2,125,15,60.51,187.5
...,...,...,...,...,...,...,...,...,...,...,...,...
20995,33-294-7116,60,1000,13,1000,9/11/2020,4,3,83.99,98.01,55.72,125.985
20996,33-294-7116,60,1000,13,1000,9/11/2020,4,3,225,73.09,55.72,337.5
20997,33-294-7116,60,1000,13,1000,9/11/2020,4,3,133.17,97.48,55.72,199.755
20998,33-294-7116,60,1000,13,1000,9/11/2020,4,3,210,51.05,55.72,315


Finally, some columns created to populate the **Orders** table will be ignored, and then save every table as a `csv` file.

In [36]:
# Saving the dataframe as .csv file for further porpuses
Customers.to_csv('Customers.csv',index=False)

# For the bookings table
Bookings = Bookings[['CustomerID', 'OrderDate', 'TableNumber']]
Bookings.to_csv('Bookings.csv',index=False)

# For the Staff table
Staff = Staff[['StaffName', 'Role', 'Salary']]
Staff.to_csv('Staff.csv',index=False)

# For the Menus table
Menus = Menus[['CourseName', 'CuisineName', 'StarterName', 'DesertName', 'Drink', 'Sides']]
Menus.to_csv('Menus.csv',index=False)

# For the DeliveryStatus table
DeliveryStatus = DeliveryStatus[['OrderDate', 'DeliveryDate']]
DeliveryStatus.to_csv('DeliveryStatus.csv',index=False)

# Saving the dataframe as .csv file for further porpuses
Orders.to_csv('Orders.csv',index=False)

### Populate the LittleLemonDB
Now that we've created the tables, we'll run a query on MySQL Workbench to populate the database, such query will look like this:

![](Populate_LittleLemonDB.png)

### Reports
Now that we've created and populated our databse, let's create some sales reports.

In [37]:
# Establish connection between Python and MySQL database via connector API
try:
    cnx = connector.connect(
        user="root", # use your own
        password="", # use your own
        database = "littlelemondb"
        )
    print("Connection between MySQL and Python is established.")
except Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

# Create cursor object to communicate with entire MySQL database
cursor = cnx.cursor()
print("Cursor is created to communicate with the MySQL using Python.")

Connection between MySQL and Python is established.
Cursor is created to communicate with the MySQL using Python.


Little Lemon needs to create a virtual table called OrdersView that focuses on OrderID, Quantity, and Cost columns within the Orders table for all orders with a quantity greater than 2.

In [38]:
cursor.execute("DROP VIEW IF EXISTS ordersview;")

# The query that creates the OrdersView view.
OrdersView = """
    CREATE VIEW OrdersView AS 
    SELECT OrderID, Quantity, Cost FROM Orders
    WHERE Quantity > 2
    ORDER BY RAND()
    LIMIT 5;
"""
# Execute the query
cursor.execute(OrdersView)

# We can see that we've created the view
cursor.execute("""SHOW FULL TABLES IN littlelemondb WHERE TABLE_TYPE LIKE 'VIEW';""")
results = cursor.fetchall()
print(cursor.column_names)
for result in results:
    print(result)

('Tables_in_littlelemondb', 'Table_type')
('ordersview', 'VIEW')


In [39]:
# Let's see some values from the previous view
cursor.execute("""SELECT * FROM OrdersView;""")

# Retrieving records
results = cursor.fetchall()

# Retrieving the column names and printing the data
print(cursor.column_names)
for result in results:
    print(result)

('OrderID', 'Quantity', 'Cost')
(11307, 3, Decimal('310.00'))
(18644, 3, Decimal('133.17'))
(3069, 3, Decimal('91.84'))
(17376, 3, Decimal('98.00'))
(1585, 3, Decimal('44.57'))


Now let's say that Little Lemon needs information from all customers with orders that cost more than $150, and the information is:

* The customerid and full name.
* The order id and cost.
* The menus name.
* Course name and starter name.

The result set should be sorted by the lowest cost amount.

In [40]:
# The query to extract the information.
more150 = """
    SELECT c.CustomerID, c.CustomerName AS FullName, o.OrderID, m.CuisineName, m.CourseName, m.StarterName, o.Cost FROM customers c
    INNER JOIN bookings b
    ON c.CustomerID = b.CustomerID
    INNER JOIN orders o
    ON b.BookingID = o.BookingID
    INNER JOIN menus m
    ON o.MenuID = m.MenuID
    WHERE o.Cost > 150
    ORDER BY o.Cost, RAND()
    LIMIT 5;
"""
# Execute the query and retrieving records
cursor.execute(more150)
results = cursor.fetchall()

# Retrieving the column names and printing the data
print(cursor.column_names)
for result in results:
    print(result)

('CustomerID', 'FullName', 'OrderID', 'CuisineName', 'CourseName', 'StarterName', 'Cost')
('37-402-2949', 'Erika Greensmith', 3042, 'Greek', 'Greek salad', 'Hummus', Decimal('168.27'))
('12-760-8688', 'Bonnee Keller', 16341, 'Italian', 'Kabasa', 'Olives', Decimal('168.27'))
('14-968-9727', 'Desdemona Dobbinson', 4791, 'Greek', 'Greek salad', 'Olives', Decimal('168.27'))
('77-254-6430', 'Renato Paulin', 12942, 'Greek', 'Kabasa', 'Minestrone', Decimal('168.27'))
('94-757-1930', 'Berny Elcoate', 8751, 'Greek', 'Pizza', 'Hummus', Decimal('168.27'))


Now, Little Lemon needs to find all menu items for which more than 2 orders have been placed.

In [41]:
# The query to extract the information.
more2orders = """
    SELECT CourseName, CuisineName FROM menus
    WHERE MenuID = ANY (SELECT MenuID FROM orders
	                    WHERE Quantity > 2)
    ORDER BY RAND()
    LIMIT 5;
"""
# Execute the query and retrieving records
cursor.execute(more2orders)
results = cursor.fetchall()

# Retrieving the column names and printing the data
print(cursor.column_names)
for result in results:
    print(result)

('CourseName', 'CuisineName')
('Greek salad', 'Italian')
('Bean soup', 'Turkish')
('Greek salad', 'Italian')
('Carbonara', 'Turkish')
('Kabasa', 'Italian')


### Procedures
Now we can move on queries and see a different tool, as procedures.

Little Lemon needs to create a procedure that displays the maximum ordered quantity in the Orders table. 
Creating this procedure will allow Little Lemon to reuse the logic implemented in the procedure easily without retyping the same code over again and again to check the maximum quantity.

In [42]:
cursor.execute("DROP PROCEDURE IF EXISTS GetMaxQuantity;")

# Creating the procedure
GetMaxQuantity = """
    CREATE PROCEDURE GetMaxQuantity()
    BEGIN
        SELECT MAX(Quantity) AS 'Max Quantity in Orders' FROM orders;
    END
"""
# Execute the query
cursor.execute(GetMaxQuantity)

# Calling the store procedure
cursor.callproc('GetMaxQuantity')

# Retrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Max Quantity in Orders']
(3,)


Little Lemon needs to create a prepared statement called GetOrderDetail. This prepared statement will help to reduce the parsing time of queries. The prepared statement should accept one input argument, the CustomerID value, from a variable. The statement should return the order id, the quantity, and the order cost from the Orders table.

In [43]:
# Creating the prepare statement
GetOrderDetail = """
    PREPARE GetOrderDetail FROM 
        'SELECT OrderID, Quantity, Cost FROM orders
        WHERE OrderID = ?';
"""
cursor.execute(GetOrderDetail)

# Setting a random input to simulate the order id
setid = """SET @id = %s;"""
OrderID = int(random.randint(1,20000))
cursor.execute(setid, (OrderID,))

# Execute the query that calls the procedure with the previous order id
cursor.execute("""EXECUTE GetOrderDetail USING @id;""")

# Retrieve values, column names and printing out the information
results = cursor.fetchall()
print(cursor.column_names)
for result in results:
    print(result)

('OrderID', 'Quantity', 'Cost')
(2269, 2, Decimal('75.00'))


Now, Little Lemon wants to create a stored procedure called CancelOrder and use this stored procedure to delete an order record based on the user input of the order id.

In [44]:
cursor.execute("DROP PROCEDURE IF EXISTS CancelOrder;")

# Creating the procedure
CancelOrder = """
    CREATE PROCEDURE CancelOrder(IN order_id INT)
    BEGIN
        DELETE FROM orders WHERE OrderID = order_id;
        SELECT CONCAT("Order ", order_id , " is cancelled") AS Confirmation;
    END
"""
# Execute the query
cursor.execute(CancelOrder)

# Calling the store procedure
cursor.callproc('CancelOrder', (16417,))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('Order 16417 is cancelled',)


Little Lemon need you to create a stored procedure called CheckBooking to check whether a table in the restaurant is already booked. The procedure should have two input parameters in the form of booking date and table number. You can also create a variable in the procedure to check the status of each table.

In [45]:
cursor.execute("DROP PROCEDURE IF EXISTS CheckBooking;")

# Creating the procedure
CheckBooking  = """
    CREATE PROCEDURE CheckBooking (IN booking_date DATE, IN table_number INT)
    BEGIN
        IF (SELECT COUNT(*) FROM bookings WHERE BookingDate = booking_date AND TableNumber = table_number) > 0 THEN
            SELECT CONCAT ('Table ', table_number , ' is already booked') AS 'Bookings status';
        ELSE
            SELECT CONCAT ('Table ', table_number , ' available') AS 'Bookings status';
        END IF;
    END
"""
# Execute the query
cursor.execute(CheckBooking)

# Calling the store procedure
cursor.callproc('CheckBooking', ('2022-05-04', 5))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Bookings status']
('Table 5 is already booked',)


In [46]:
# Now if we try with a booking that doesn't exist
cursor.callproc('CheckBooking', ('2023-07-04', 8))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Bookings status']
('Table 8 available',)


Little Lemon need to verify a booking, and decline any reservations for tables that are already booked.

Since integrity is not optional, Little Lemon need to ensure that every booking attempt includes these verification and decline steps. However, implementing these steps requires a stored procedure and a transaction. 

To implement these steps, we need to create a new procedure called AddValidBooking. This procedure must use a transaction statement to perform a rollback if a customer reserves a table that’s already booked.  

* The procedure should include three input parameters in the form of customer id, booking date and table number.
* It also requires at least one variable and should begin with a START TRANSACTION statement.
* Your INSERT statement must add a new booking record using the input parameter's values.
* Use an IF ELSE statement to check if a table is already booked on the given date. 

If the table is already booked, then rollback the transaction. If the table is available, then commit the transaction. 

In [47]:
cursor.execute("DROP PROCEDURE IF EXISTS AddValidBooking;")

# Creating the procedure
AddValidBooking  = """
    CREATE PROCEDURE AddValidBooking (IN customer_id varchar(255), IN booking_date DATE, IN table_number INT)
    BEGIN
        START TRANSACTION;
            INSERT INTO bookings(CustomerID, BookingDate, TableNumber) VALUES(customer_id, booking_date, table_number);
        IF (SELECT COUNT(*) FROM bookings WHERE BookingDate = booking_date AND TableNumber = table_number) = 0 
            OR NULL = (SELECT COUNT(*) FROM bookings WHERE BookingDate = booking_date AND TableNumber = table_number) THEN
            COMMIT;
            SELECT CONCAT('Table number ', table_number, ' is now booked for the ', booking_date, ' booking successful') AS 'Booking status';
        ELSE
            ROLLBACK;
			SELECT CONCAT('Table number ', table_number, ' is already booked - booking cancelled') AS 'Booking status';
        END IF;
    END
"""
# Execute the query
cursor.execute(AddValidBooking)

# Calling the store procedure
cursor.callproc('AddValidBooking', ('97-003-2495','2022-05-04',5))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Booking status']
('Table number 5 is already booked - booking cancelled',)


Let's create a new procedure called AddBooking to add a new table booking record. The procedure should include four input parameters in the form of the following bookings parameters:

* Booking id, 
* Customer id, 
* Booking date,
* Table number.

In [48]:
cursor.execute("DROP PROCEDURE IF EXISTS AddBooking;")

# Creating the procedure
AddBooking  = """
    CREATE PROCEDURE AddBooking (IN booking_id INT, IN customer_id VARCHAR(255), IN booking_date VARCHAR(255), IN table_number INT)
    BEGIN
        DECLARE subqu INT;
        SELECT COUNT(*) INTO subqu FROM bookings 
			WHERE BookingID = booking_id AND CustomerID = customer_id AND BookingDate = booking_date AND TableNumber = table_number;
        IF subqu = 0 THEN
			INSERT INTO bookings(BookingID, CustomerID, BookingDate, TableNumber) VALUES(booking_id, customer_id, booking_date, table_number);
			SELECT CONCAT('New booking added') AS 'Confirmation';
        ELSE
			SELECT CONCAT('The customer ', customer_id, ' already booked this table for this date - Booking cancelled') AS 'Confirmation';
        END IF;
    END
"""
# Execute the query
cursor.execute(AddBooking)

# Calling the store procedure
cursor.callproc('AddBooking', (1199,'79-877-0995','2022-08-14',13))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('The customer 79-877-0995 already booked this table for this date - Booking cancelled',)


In [49]:
# Now if we try with a booking that does exist
cursor.callproc('AddBooking', (1199,'79-877-0995','2022-08-14',13))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('The customer 79-877-0995 already booked this table for this date - Booking cancelled',)


Little Lemon need you to create a new procedure called UpdateBooking that they can use to update existing bookings in the booking table.
The procedure should have two input parameters in the form of booking id and booking date. 

In [50]:
cursor.execute("DROP PROCEDURE IF EXISTS UpdateBooking;")

# Creating the procedure
UpdateBooking  = """
    CREATE PROCEDURE UpdateBooking (IN booking_id INT, IN booking_date VARCHAR(255))
    BEGIN
        DECLARE subqu INT;
        SELECT COUNT(*) INTO subqu FROM bookings 
			WHERE BookingID = booking_id;
        IF subqu = 1 THEN
            UPDATE bookings SET BookingDate = booking_date WHERE BookingID = booking_id;
			SELECT CONCAT('Booking ', booking_id , ' updated') AS 'Confirmation';
        ELSE
			SELECT CONCAT('Booking ', booking_id , ' does not exist') AS 'Confirmation';
        END IF;
    END
"""
# Execute the query
cursor.execute(UpdateBooking)

# Calling the store procedure
cursor.callproc('UpdateBooking', (885,'2022-08-14'))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('Booking 885 updated',)


In [51]:
# Now if we try with a booking that doesn't exist
cursor.callproc('UpdateBooking', (1001,'2022-08-14'))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('Booking 1001 does not exist',)


Little Lemon needs  to create a new procedure called CancelBooking that they can use to cancel or remove a booking.

In [52]:
cursor.execute("DROP PROCEDURE IF EXISTS CancelBooking;")

# Creating the procedure
CancelBooking  = """
    CREATE PROCEDURE CancelBooking (IN booking_id INT)
    BEGIN
        DECLARE subqu INT;
        SELECT COUNT(*) INTO subqu FROM bookings 
			WHERE BookingID = booking_id;
        IF subqu = 1 THEN
            DELETE FROM bookings WHERE BookingID = booking_id;
			SELECT CONCAT('Booking ', booking_id , ' cancelled') AS 'Confirmation';
        ELSE
			SELECT CONCAT('Booking ', booking_id , ' does not exist') AS 'Confirmation';
        END IF;
    END
"""
# Execute the query
cursor.execute(CancelBooking)

# Calling the store procedure
cursor.callproc('CancelBooking', (1000,))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('Booking 1000 cancelled',)


In [53]:
# Now if we try with a booking that doesn't exist
cursor.callproc('CancelBooking', (1001,))

# Rretrieving records in a dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data
for data in dataset:
    print(data)

['Confirmation']
('Booking 1001 does not exist',)


In [54]:
# Let's close the cursor and the connection
if cnx.is_connected():
   cursor.close()
   print("The cursor is closed.")
   cnx.close()
   print("MySQL connection is closed.")
else:
   print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.


### A little data analysis

Once we clean the data, populate the database and create the desired procedures, we can make some graphics for any data analysis.
The data analysis will be done for the United States and will be necessary to create an additional field called "profit", which will be the difference between cost and sale.

The data analysis will be:
* A bar chart that shows customers sales and filter data based on sales with at least $70. ![](Customer_Sales.png)

* A line chart to show the sales trend from 2019 to 2022. ![](Profit_chart.png)
* A Bubble chart of sales for all customers. The chart should show the names of all customers. Once you roll over a bubble, the chart should show the name, profit and sale. ![](Sales_Bubble_Chart.png)
* A Bar chart that shows the sales of the Turkish, Italian and Greek cuisines. We need to display sales data for 2020, 2021, and 2022 only. Each bar should display the profit of each cuisine. ![](Cuisine_Sales_profit.png)
* an interactive dashboard that combines the Bar chart called Customers sales and the Sales Bubble Chart. Once you click a bar, and roll over the related bubble, the name, sales and profit figures should be displayed in the Bubble chart as shown below. ![](Dashboard.png)

### Conclusion
As we could see, a file with enough information could be used to create and populate a database; although the conecction to that database was done throught the use of a MuSQL API, the conection was secure enough to make some queries and even create some procedures. At the end we use the same file to make a little data analysis, but we could also use the files created to populate the database.